In [ ]:
import os
import warnings
from pathlib import Path
import subprocess
import rasterio as rio
import pandas as pd
import geopandas as gpd
import pickle
import xarray as xr
import rioxarray as rioxr
import numpy as np
import plotly.express as px
from skimage import morphology
from skimage.measure import label
from skimage.metrics import structural_similarity as ssim
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

## Set up paths and read in files for subsequent processing

In [ ]:
#Main input filepath. All other files and folders should be nested under this one (aside from the road geom file specified below)
inpath = Path("../data/processed/outputs_BeitBridge_2022-01-01_to_2023-03-27")

#Geometry file for clipping tdi outputs
road_geom_file = "../data/processed/beitbridge_road_mask_v2.gpkg"

img_path = inpath.joinpath("Beitbridge_PlanetScope") #Subfolder containing images downloaded from skywatch api

outpath_aligned = inpath.joinpath("Processed_Images") #Output path for co-aligned images
outpath_aligned.mkdir(exist_ok=True)

outpath_workdir = inpath.joinpath("TDI_Outputs") #Output directory for TDI file outputs
outpath_workdir.mkdir(exist_ok=True)

img_df_csv = inpath.joinpath("search_df_2022-01-01_to_2023-03-27_filtered.csv") #CSV file generated from dataframe created in the skywatch api notebook
img_df = pd.read_csv(img_df_csv)
img_df.drop(columns=['Unnamed: 0'], inplace=True)


## Stage 1: Align images
This section of the notebook is to take the PlanetScope images downloaded using the skywatch api notebook and resample them to a common pixel grid.
Note: this step should only need to be run once.

In [ ]:
#Set up a base image to resample everything to match. It selects the first image with 100% AOI coverage to use as the base image
base_imgname = img_df.loc[img_df["aoi_coverage"] == 100].iloc[0]["product_name"]
base_img_file = img_path.joinpath(base_imgname, f"{base_imgname}_analytic.tif")
base_img = rio.open(base_img_file)


In [ ]:
def resample_image(base_img, warp_imgname, out_name):
    """
    Function to resample an image to match the base image

    Params:
    ---------------------
    base_img: rasterio dataset
        base image to be aligned to
    warp_imgname: pathlib Path
        path to the image to be resampled.
    out_name: pathlib Path
        output path for resampled image
    """
    
    srs = base_img.crs.to_string()
    pixelsize_x, pixelsize_y = base_img.res
    bbox = base_img.bounds

    #First step: create temp file using gdalwarp which resamples the target image to match the base image
    cmd = f"gdalwarp -t_srs {srs} -tap -tr {pixelsize_x} {pixelsize_y} -r near -te {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} -te_srs {srs} -of vrt {warp_imgname} {out_name}.vrt -overwrite"
    subprocess.check_output(cmd, shell=True)

    #Second step: use gdal_translate to compress the temp file into the final output
    cmd2 = f"gdal_translate -co compress=LZW {out_name}.vrt {out_name}.tif"
    subprocess.check_output(cmd2, shell=True)

#Iterate through all the images in the img_df dataframe and resample to match the base image
for index, row in img_df.iterrows():
    img_name = row["product_name"]    
    img_file = img_path.joinpath(img_name, f"{img_name}_analytic.tif")
    img_processedname = outpath_aligned.joinpath(f"{img_name}_aligned")
    resample_image(base_img, img_file, img_processedname)
    
    os.remove(f"{img_processedname}.vrt") #Delete temp file
    print(f"finished resampling image: {img_processedname}")    

## Stage 2: Compute median image
This section of the notebook is used to create a median image from all the aligned images. Note: this section should only need to be run once.

In [ ]:
#Iterate through all co-registered images and stack them into a big xarray
datasets = []
date_list = []
for index, row in img_df.iterrows():
    img_name = row["product_name"]
    img_date = row['datestr']
    img_file = outpath_aligned.joinpath(f"{img_name}_aligned.tif")
    img_arr = rioxr.open_rasterio(img_file)
    datasets.append(img_arr)
    date_list.append(img_date)

time_var = xr.Variable('time', date_list)
combined_arr = xr.concat(datasets, dim=time_var)

In [ ]:
#Calculate the pixel and band-wise median from the stacked xarray
median_arr = combined_arr.median(dim='time')

In [ ]:
#Outputs temporary median file 
temp_median_file = outpath_aligned.joinpath("median_temp.tif")
median_arr.rio.to_raster(raster_path=temp_median_file)

#Compresses temp median file into final median file
median_file = outpath_aligned.joinpath("median_image.tif")
cmd = f"gdal_translate -a_nodata 0 -co compress=LZW {temp_median_file} {median_file}"
subprocess.check_output(cmd, shell=True)
os.remove(temp_median_file)

## Stage 3: Tophat filtering and detection
This section of the notebook performs the tophat filtering and outputs the final vehicle detections. If Stage 1 and 2 have already been run, start here.

### Function definitions

In [ ]:
def export_xarray(export_arr, ref_arr, xr_filename):
    """
    Function used to convert a numpy array to an xarray and export it
    
    Params:
    ---------------------
    export_arr: numpy array
        array to be exported
    ref_arr: xarray data array
        reference xarray to be used to set up the output
    xr_filename: pathlib Path
        filepath for the output file
    """

    coords = {'x': ref_arr.coords['x'].to_numpy(), 'y': ref_arr.coords['y'].to_numpy()}
    export_xr = xr.DataArray(export_arr, coords=coords, dims=('y', 'x'))

    export_xr.rio.write_crs(4326, inplace=True)
    export_xr.rio.to_raster(raster_path=xr_filename) 

def get_kernel(size):
    """
    Function defines kernels for multi-directional tophat filtering based on specified kernel size.

    Params:
    ---------------------
    size: int
        size for the tophat kernel. Options are 3, 5, and 7 currently.
    
    Returns:
    ---------------------
    kernel_dict: dict
        dictionary with tophat kernels
    """
    
    if size == 3:
        kernel_0 = np.array([[0,0,0],
                             [1,1,1],
                             [0,0,0]])
        kernel_45 = np.array([[1,0,0],
                              [0,1,0],
                              [0,0,1]])
        kernel_90 = np.array([[0,1,0],
                              [0,1,0],
                              [0,1,0]])
        kernel_135 = np.array([[0,0,1],
                               [0,1,0],
                               [1,0,0]])
    
    if size == 5:
        kernel_0 = np.array([[0,0,0,0,0],
                             [0,0,0,0,0],
                             [1,1,1,1,1],
                             [0,0,0,0,0],
                             [0,0,0,0,0]])
        kernel_45 = np.array([[1,0,0,0,0],
                              [0,1,0,0,0],
                              [0,0,1,0,0],
                              [0,0,0,1,0],
                              [0,0,0,0,1]])
        kernel_90 = np.array([[0,0,1,0,0],
                              [0,0,1,0,0],
                              [0,0,1,0,0],
                              [0,0,1,0,0],
                              [0,0,1,0,0]])
        kernel_135 = np.array([[0,0,0,0,1],
                               [0,0,0,1,0],
                               [0,0,1,0,0],
                               [0,1,0,0,0],
                               [1,0,0,0,0]])
    if size == 7:
        kernel_0 = np.array([[0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0],
                             [1,1,1,1,1,1,1],
                             [0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0]])
        kernel_45 = np.array([[1,0,0,0,0,0,0],
                              [0,1,0,0,0,0,0],
                              [0,0,1,0,0,0,0],
                              [0,0,0,1,0,0,0],
                              [0,0,0,0,1,0,0],
                              [0,0,0,0,0,1,0],
                              [0,0,0,0,0,0,1]])
        kernel_90 = np.array([[0,0,0,1,0,0,0],
                              [0,0,0,1,0,0,0],
                              [0,0,0,1,0,0,0],
                              [0,0,0,1,0,0,0],
                              [0,0,0,1,0,0,0],
                              [0,0,0,1,0,0,0],
                              [0,0,0,1,0,0,0]])
        kernel_135 = np.array([[0,0,0,0,0,0,1],
                               [0,0,0,0,0,1,0],
                               [0,0,0,0,1,0,0],
                               [0,0,0,1,0,0,0],
                               [0,0,1,0,0,0,0],
                               [0,1,0,0,0,0,0],
                               [1,0,0,0,0,0,0]])

    kernel_dict = {'kernel_0': kernel_0, 'kernel_45': kernel_45, 'kernel_90': kernel_90, 'kernel_135': kernel_135}

    return kernel_dict

def calc_tophat_band(band_arr, kernel_size, tophat_type):
    """
    Computes multiple tophat filters for a specific difference band (difference between target image
    band and median image band). Returns the minimum tophat value from the multi-directional filtering.

    Params:
    ---------------------
    band_arr: xarray data array
        array for the specified band
    kernel_size: int
        size of kernel to use for tophat filtering
    tophat_type: str
        type of tophat filter to use. Options are "white" and "black"

    Returns:
    ---------------------
    min_tophat: xarray data array
        pixel-wise minimum of all tophat kernels
    """

    kernel_dict = get_kernel(size=kernel_size)
    
    tophat_arrlist = []

    for key, val in kernel_dict.items():
        if tophat_type=='white':
            tophat_arr = morphology.white_tophat(band_arr, footprint=val)
        if tophat_type=='black':
            tophat_arr = morphology.black_tophat(band_arr, footprint=val)
        tophat_arrlist.append(tophat_arr)

    tophat_combine = np.array(tophat_arrlist)
    tophat_combine_coords = {'x': band_arr.coords['x'].to_numpy(), 'y': band_arr.coords['y'].to_numpy()}
    tophat_combine_xr = xr.DataArray(tophat_combine, coords=tophat_combine_coords, dims=('band', 'y', 'x'))
    min_tophat = tophat_combine_xr.min(dim='band')

    return min_tophat

def calc_diff(band_arr, med_arr, method):
    """
    Computes difference between band array from target image and corresponding
    band from median image. Two methods are possible: simple pixel differencing or the
    structural similarity index (ssim). Note that ssim was tested briefly but is not the 
    operational method.

    Params:
    ---------------------
    band_arr: xarray data array
        band array from target image
    med_arr: xarray data array
        band array from median image
    method: str
        Type of difference method to apply. Options are "diff" and "ssim"
    
    Returns:
    ---------------------
    diff_arr: numpy array
        array of pixel differences between band_arr and med_arr
    """

    if method == 'diff':
        diff_arr = band_arr.to_numpy() - med_arr.to_numpy()
    if method == 'ssim':
        ssim_score, diff_arr = ssim(med_arr.to_numpy(), band_arr.to_numpy(), win_size=3, full=True)

    return diff_arr

def run_tophat_filter(img_arr, median_arr, band_subset, diff_method, kernel_size):
    """
    Function to perform tophat filtering on an image. Iterates through image bands, computes
    difference between the target image band and the median image band, and runs tophat filtering
    function (runs both "black" and "white" tophat filters). Once tophat filtering has been completed
    for all bands, the maximum of all of the individual tophat filters for all bands is computed for each
    tophat filter type (black and white).

    Params:
    ---------------------
    img_arr: xarray data array
        xarray data array for the target image
    median_arr: xarray data array
        xarray data array for the median image
    band_subset: str
        subset of image bands to use. Options are "allbands" to use all 8 superdove bands, or 
        "4band" to use only the BGRN bands.
    diff_method: str
        Type of difference method to apply. Options are "diff" and "ssim"
    kernel_size: int
        size of kernel to use for tophat filtering
    
    Returns:
    ---------------------
    tophat_dict: dict
        dictionary containing numpy arrays for the two final tophat filters for the target image (black and white)
    """

    tophat_typelist=['black', 'white']
    band_index_dict = {'allbands': [0,1,2,3,4,5,6,7], '4band': [1,3,5,7]}
    band_indices = band_index_dict[band_subset]

    tophat_dict = {}
    for tophat_type in tophat_typelist:
        tophat_minlist = []
        for i in range(0, len(img_arr)):
            if i not in band_indices:
                continue
            band_xr = img_arr[i]
            diff_arr = calc_diff(band_xr, median_arr[i], diff_method)
            diff_xr = xr.DataArray(diff_arr, coords=band_xr.coords, dims=band_xr.dims, attrs=band_xr.attrs)
            band_tophat = calc_tophat_band(diff_xr, kernel_size, tophat_type)
            tophat_minlist.append(band_tophat)

        min_tophat_combine_xr = xr.concat(tophat_minlist, dim='band')
        tophat_final = min_tophat_combine_xr.max(dim='band')
        tophat_dict[tophat_type] = tophat_final
        
        # tophat_finalfile = outpath_workdir.joinpath(f"{select_date}_{diff_method}_method_tophat_{tophat_type}_kernel_{kernel_size}_{band_subset}.tif")
        # tophat_final.rio.write_crs(4326, inplace=True)
        # tophat_final.rio.to_raster(raster_path=tophat_finalfile) 

    return tophat_dict

def combine_filters(tophat_white_arr, tophat_black_arr):
    """
    Combine the black and white tophat filters for the target image into a single filter which is
    the pixel-wise maximum of both.

    Params:
    ---------------------
    tophat_white_arr: numpy array
        white tophat filter array
    tophat_black_arr: numpy array
        black tophat filter array

    Returns:
    ---------------------
    max_tophat: xarray data array
        combined tophat filter array
    """
    
    combined_tophat = xr.concat([tophat_white_arr, tophat_black_arr], dim='band')
    max_tophat = combined_tophat.max(dim='band')
    max_tophat.rio.write_crs(4326, inplace=True)

    return max_tophat

def clip_tophat(max_tophat, clip_geom):
    """
    Clips tophat filter to road and parking area geometry

    Params:
    ---------------------
    max_tophat: xarray data array
        tophat filter array
    clip_geom: geopandas geodataframe
        geodataframe containing the clipping geometry

    Returns:
    ---------------------
    clipped: xarray data array
        clipped tophat filter array
    """

    clipped = max_tophat.rio.clip(clip_geom.geometry.values, clip_geom.crs)

    return clipped

def apply_threshold(max_tophat, threshold):
    """
    Function to apply a threshold to a tophat filter array to create a boolean array.

    Params:
    ---------------------
    max_tophat: xarray data array
        tophat filter array
    threshold: float
        threshold to apply to tophat array

    Returns:
    tophat_bool: numpy array
        boolean tophat array
    ---------------------
    """
    
    tophat_bool = xr.where(max_tophat >= threshold, 1, 0)
    tophat_bool = tophat_bool.astype(np.int8).to_numpy()
    
    return tophat_bool

def sieve_bool_filter(tophat_bool, sieve_thresh):
    """
    Apply sieving to boolean tophat array to filter out small objects.

    Params:
    ---------------------
    tophat_bool: numpy array
        boolean tophat array
    sieve_thresh: int
        threshold to use for sieving. Objects smaller than the threshold are removed.

    Returns:
    ---------------------
    tophat_sieve: numpy array
        sieved boolean tophat array
    """
    
    labelled = label(tophat_bool)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        tophat_sieve = morphology.remove_small_objects(labelled, min_size=sieve_thresh, connectivity=1)
        
    tophat_sieve = np.where(tophat_sieve > 0, 1, 0)
    tophat_sieve = tophat_sieve.astype(np.int8)
    
    return tophat_sieve

def threshold_tophat(clipped_tophat, min_thresh, sieve_thresh):
    """
    Function to apply thresholding to the image tophat filters to create boolean detection arrays.
    Initiallty uses the maximum value from the tophat array and iteratively reduces until it reaches 
    the specified minimum threshold value. After each iteration the results from the previous iteration 
    is subtracted from the current one, and sieving is applied to remove small objects. The sieved 
    filter array is then added together with the previous results to create a combined array.
    
    Note this iterative approach does not seem to add much value over just using the minimum value right 
    from the get-go, but does reduce the number of stray pixels and makes the detections possibly a bit more cohesive.
    The iterative approach was implemented to adhere to the workflow outlined in Chen et al. 2021

    Params:
    ---------------------
    clipped_tophat: xarray data array
        clipped tophat filter array
    min_thresh: float
        minimum threshold value to use to create boolean vehicle detection array
    sieve_thresh: int
        minimum object size (in pixels). Objects smaller than the specified size are removed

    Returns:
    ---------------------
    final_bool: numpy array
        boolean vehicle detection array

    """
    
    max_thresh = clipped_tophat.max()
    current_thresh = max_thresh
    thresh_multiplier = 0.9

    if current_thresh < min_thresh:
        print(f'Max tophat value less than the specified minimum threshold setting: max tophat val = {max_thresh}; min threshold = {min_thresh}')
        raise KeyboardInterrupt

    first_iter = True
    while current_thresh >= min_thresh:
        if first_iter:
            tophat_bool = apply_threshold(clipped_tophat, current_thresh)
            tophat_bool = sieve_bool_filter(tophat_bool, sieve_thresh)
            previous_bool = np.copy(tophat_bool)
            first_iter = False
        else:
            tophat_bool = apply_threshold(clipped_tophat, current_thresh)
            tophat_bool = tophat_bool - previous_bool
            tophat_bool = sieve_bool_filter(tophat_bool, sieve_thresh)
            previous_bool = previous_bool + tophat_bool
        
        current_thresh = current_thresh * thresh_multiplier

    if current_thresh < min_thresh:
        current_thresh = min_thresh
        tophat_bool = apply_threshold(clipped_tophat, current_thresh)
        tophat_bool = tophat_bool - previous_bool
        tophat_bool = sieve_bool_filter(tophat_bool, sieve_thresh)
        previous_bool = previous_bool + tophat_bool

    final_bool = np.copy(previous_bool)

    return final_bool

def calc_tdi(clipped_band, detection_arr):
    """
    Calculates the Traffic Density Index (TDI) by dividing the number of vehicle detection pixels
    by the total number of pixels within the road and parking lot geometry mask.

    Params:
    ---------------------
    clipped_band: xarray data array
        band from target image clipped to road geometry
    detection_arr: numpy array
        boolean vehicle detection array

    Returns:
    ---------------------
    tdi: float
        computed tdi value
    """

    total_pixels = np.count_nonzero(clipped_band.to_numpy())
    detection_pixels = np.count_nonzero(detection_arr)

    tdi = (detection_pixels / total_pixels) * 100.0

    return tdi

### Tophat filtering code execution begins here

In [ ]:
#Read in median image file
median_file = outpath_aligned.joinpath("median_image.tif")
median_arr = rioxr.open_rasterio(median_file)

In [ ]:
#Create output directory for the tophat rasters and boolean vehicle detection rasters
tophat_outpath = outpath_workdir.joinpath("tophat_rasters")
tophat_outpath.mkdir(exist_ok=True)

detection_outpath = outpath_workdir.joinpath("detection_rasters")
detection_outpath.mkdir(exist_ok=True)

In [ ]:
#To process a single image, set a date in the "select_date" variable. Otherwise leave it set to None to process all images
select_date = None
# select_date = "2022-12-24"

#Read in road clipping geometry
road_geom_gpd = gpd.read_file(road_geom_file)


band_subset = 'allbands' #select subset of image bands or run all bands. Options are 'allbands' to use all bands, or '4band' to only process BGRN
diff_method = 'diff' #Method for comparing image with median image. 'diff' uses simple pixel differences, while 'ssim' uses the structural similarity index
kernel_size=7 #size of kernel to use for tophat filtering. Options are 3, 5, and 7
sieve_thresh = 5 #sieve threshold for filtering boolean detection arrays. Groups of detection pixels smaller than the threshold are removed
min_thresh=0.015 #minimum threshold used for vehicle detections from tophat filter array. Program will iteratively use smaller threshold values down to the value specified here.

tdi_list = [] #list of TDI values. Each TDI value from each image is appended to this list

#Iterate through all images in img_df and run vehicle detection and tdi calculation. If "select_date" variable set, will only process selected image
for index, content in img_df.iterrows():

    img_name = content["product_name"]
    img_date = content['datestr']

    if select_date and img_date != select_date:
        continue

    print(f'Calculating TDI for image: {img_name}')

    img_file = outpath_aligned.joinpath(f"{img_name}_aligned.tif")
    img_arr = rioxr.open_rasterio(img_file)
    
    #Create tophat filters for target image
    tophat_dict = run_tophat_filter(img_arr, median_arr, band_subset, diff_method, kernel_size)

    #Combine white and black tophat filters and save to file
    max_tophat = combine_filters(tophat_dict['white'], tophat_dict['black'])
    # tophat_combined_fname = outpath_workdir.joinpath(f"{img_name}_{diff_method}_method_kernel_{kernel_size}_tophat.tif")
    tophat_tempname = tophat_outpath.joinpath(f"{img_name}_tophat_temp.tif")
    tophat_combined_fname = tophat_outpath.joinpath(f"{img_name}_tophat_raster.tif")
    max_tophat.rio.write_crs(4326, inplace=True)
    max_tophat.rio.to_raster(raster_path=tophat_tempname) 
    cmd = f"gdal_translate -co compress=LZW {tophat_tempname} {tophat_combined_fname}"
    subprocess.check_output(cmd, shell=True)

    #Clip tophat array to road geometry
    clipped_tophat = clip_tophat(max_tophat, road_geom_gpd)
    
    #Create boolean vehicle detection arry from clipped tophat array
    final_bool = threshold_tophat(clipped_tophat, min_thresh, sieve_thresh)

    #Output boolean vehicle detection array
    # final_bool_fname = outpath_workdir.joinpath(f"{img_name}_{diff_method}_method_kernel_{kernel_size}_minthresh_{min_thresh}_sieve_{sieve_thresh}.tif")
    bool_tempname = detection_outpath.joinpath(f"{img_name}_vehicle_detection_temp.tif")
    final_bool_fname = detection_outpath.joinpath(f"{img_name}_vehicle_detection_raster.tif")
    export_xarray(final_bool, clipped_tophat, bool_tempname)
    cmd = f"gdal_translate -a_nodata 0 -co compress=LZW {bool_tempname} {final_bool_fname}"
    subprocess.check_output(cmd, shell=True)

    #Calculate TDI
    clipped_img_band = img_arr[1].rio.clip(road_geom_gpd.geometry.values, road_geom_gpd.crs)
    tdi = calc_tdi(clipped_img_band, final_bool)
    tdi_list.append(tdi)

    os.remove(tophat_tempname)
    os.remove(bool_tempname)

### Outputting TDI to a CSV file and creating chart. This is the end of the current process

In [ ]:
tdi_df = img_df.copy()

tdi_df['tdi'] = tdi_list

tdi_pickle = outpath_workdir.joinpath("TDI_DF_2022-01-01_to_2023-03-27.pkl")
tdi_df.to_pickle(tdi_pickle)

tdi_csv_file = outpath_workdir.joinpath("TDI_DF_2022-01-01_to_2023-03-27.csv")
tdi_df.to_csv(tdi_csv_file)



In [ ]:
# tdi_pickle = outpath_workdir.joinpath("TDI_DF_2022-01-01_to_2023-03-27.pkl")
# with open(tdi_pickle, "rb") as f:
#     tdi_df = pickle.load(f)

In [ ]:
# tdi_df.nsmallest(5, 'tdi')

In [ ]:
# tdi_df.nlargest(5, 'tdi')

In [ ]:
#Creating TDI time series line chart

tdi_fig = px.line(tdi_df, x='date', y='tdi', color_discrete_map={'tdi': 'red'})
tdi_fig.update_traces(line_color='red', line_width=2)

fignamestr = f"TDI_Chart_2022-01-01_to_2023-03-27"
tdi_out_png = outpath_workdir.joinpath(f"{fignamestr}.png")
tdi_out_html = outpath_workdir.joinpath(f"{fignamestr}.html")
tdi_fig.write_image(tdi_out_png)
tdi_fig.write_html(tdi_out_html)
tdi_fig.show()

## MISC TESTING AREA IGNORE FOR NOW

In [ ]:
# select_df = img_df.loc[img_df["datestr"] == select_date].iloc[0]
# img_name = select_df["product_name"]
# img_date = select_df['datestr']
# img_file = outpath_aligned.joinpath(f"{img_name}_aligned.tif")
# img_arr = rioxr.open_rasterio(img_file)


In [ ]:
# tophat_combined_fname = outpath_workdir.joinpath(f"{select_date}_{diff_method}_method_kernel_{kernel_size}_combinedtophat.tif")
# max_tophat.rio.write_crs(4326, inplace=True)
# max_tophat.rio.to_raster(raster_path=tophat_combined_fname) 

In [ ]:
# bool_thresh = 0.01
# sieve_thresh = 2
# tophat_bool = threshold_filter(clipped_tophat, bool_thresh, sieve_thresh)
# tophat_bool_fname = outpath_workdir.joinpath(f"{select_date}_{diff_method}_method_kernel_{kernel_size}_boolthresh_{bool_thresh}_sieve_{sieve_thresh}_clipped.tif")
# export_xarray(tophat_bool, clipped_tophat, tophat_bool_fname)
# tophat_bool.rio.write_crs(4326, inplace=True)
# tophat_bool.rio.to_raster(raster_path=tophat_bool_fname) 

## Radiometric normalization test section

In [ ]:
## TESTING RAD NORMALIZATION
# def create_rad_regmodel(band_arr, med_arr):
#     diff_arr = np.abs(band_arr - med_arr)

#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         percent_diff_arr = np.divide(diff_arr, med_arr) * 100.0

#     percent_val = np.nanpercentile(percent_diff_arr, 25.0)
#     low_diff_arr = np.where(percent_diff_arr > percent_val, np.nan, percent_diff_arr)

#     band_arr_trunc = np.where(np.isnan(low_diff_arr), np.nan, band_arr)
#     med_arr_trunc = np.where(np.isnan(low_diff_arr), np.nan, med_arr)

#     x = med_arr_trunc.reshape((-1,1))
#     y = band_arr_trunc.reshape((-1,1))
#     x = x[~np.isnan(x)].reshape((-1,1))
#     y = y[~np.isnan(y)].reshape((-1,1))

#     x_ = PolynomialFeatures(degree=1, include_bias=False).fit_transform(x)
#     model = LinearRegression().fit(x_, y)

#     slope = round(float(model.coef_[0]), 5)
#     intercept = round(float(model.intercept_[0]), 5)

#     band_shape = band_arr.shape
#     band_reshape = band_arr.reshape(-1,1)
#     x_ = PolynomialFeatures(degree=1, include_bias=False).fit_transform(band_reshape)
#     pred_flat = model.predict(x_)
#     pred_arr = pred_flat.reshape(band_shape)

#     return pred_arr
#     # return slope, intercept
#     # return model


In [ ]:
# band_indices = [0,1,2,3,4,5,6,7]

# # df_list = []

# for index, content in img_df.iterrows():
#     if index != 91:
#         continue
    
#     img_name = content["product_name"]
#     img_date = content['datestr']
#     img_file = outpath_aligned.joinpath(f"{img_name}_aligned.tif")
#     img_arr = rioxr.open_rasterio(img_file)

#     # img_dict = {}
#     # img_dict['img_name'] = img_name
#     pred_arr_list = []
#     for band_index in band_indices:
#         band_arr = img_arr[band_index].to_numpy()
#         med_arr = median_arr[band_index].to_numpy()

#         pred_arr = create_rad_regmodel(band_arr, med_arr)

#         pred_arr_list.append(pred_arr)

#         # slope, intercept = create_rad_regmodel(band_arr, med_arr)

#         # img_dict[f'b{band_index+1}_slope'] = slope
#         # img_dict[f'b{band_index+1}_intercept'] = intercept

#     # img_reg_df = pd.DataFrame(img_dict, index=[0], columns=['img_name', 'b1_slope', 'b2_slope', 'b3_slope', 'b4_slope', 'b5_slope', 'b6_slope', 'b7_slope', 'b8_slope', 'b1_intercept', 'b2_intercept', 'b3_intercept', 'b4_intercept', 'b5_intercept', 'b6_intercept', 'b7_intercept', 'b8_intercept'])
#     # df_list.append(img_reg_df)

#     radcor_arr = np.array(pred_arr_list)
#     radcor_fname = outpath_workdir.joinpath(f"{img_date}_radcor.tif")
    
#     radcor_xr = xr.DataArray(radcor_arr, coords=img_arr.coords, dims=img_arr.dims)
#     radcor_xr.rio.write_crs(4326, inplace=True)
#     radcor_xr.rio.to_raster(raster_path=radcor_fname) 

# regression_df = pd.concat(df_list)
# regression_df.reset_index(inplace=True, drop=True)

In [ ]:
# regression_df['b2_slope'].idxmax()

In [ ]:
# regression_df.iloc[91]